# Calculations for FOV

This notebook goes into the caculations for the Field of View for my camera. This should allow me to calculate a box that I can put over my star maps to see what I should be able to see.

FOV in arcseconds of a degree = $\dfrac{Chip Size}{Focal Length}$ * $\dfrac{360^{\circ}}{2\pi}$ * 60$\dfrac{arcminutes}{1^{\circ}}$ * 60  $\dfrac{arcseconds}{arcminute}$

## ASI482MC

So for the ASI482MC:
* Resolution: 2.07Mega Pixel,1920 x 1080
* Pixel Size: 5.8um x 5.8um
* Dimension of sensor: 11.13mm x 6.26mm

In [1]:
# My telescope has 1600mm focal length
focal_length = 1600

In [2]:
import math

FOV_x = (11.13/focal_length) * (360/(2*math.pi)) * 60 * 60
FOV_x

1434.829558456364

In [3]:
FOV_y = (6.26/focal_length) * (360/(2*math.pi)) * 60 * 60
FOV_y

807.0110544417645

So the theroretical FOV for the ASI482MC is: 1435 x 807 arcseconds

We will want to know the field of view per pixel for later.

In [4]:
ASI482MC_FOV_pixel = (0.0058/focal_length) * (360/(2*math.pi)) * 60 * 60
ASI482MC_FOV_pixel

0.7477099226457243

## ASI120MC-S

So for the ASI120MC-S:
* Resolution: 1.2Mega Pixels 1280×960
* Pixel Size: 3.75um x 3.75um
* Dimension of sensor: 4.8mm x 3.6mm

In [5]:
FOV_x = (4.8/focal_length) * (360/(2*math.pi)) * 60 * 60
FOV_x

618.7944187412891

In [6]:
FOV_y = (3.6/focal_length) * (360/(2*math.pi)) * 60 * 60
FOV_y

464.09581405596685

So the theroretical FOV for the ASI120MC-S is: 619 x 464 arcseconds

We will want to know the field of view per pixel for later.

In [7]:
ASI120MC_S_FOV_pixel = (0.00375/focal_length) * (360/(2*math.pi)) * 60 * 60
ASI120MC_S_FOV_pixel

0.48343313964163204

## 2.5mm 170 degree "All Sky Lens" attatchment

The ASI120MC-S has an "all sky" lens attachment.  

FOV in arcseconds of a degree = degrees * 60$\dfrac{arcminutes}{1^{\circ}}$ * 60  $\dfrac{arcseconds}{arcminute}$


In [8]:
ALL_SKY_FOV = 170*60*60
ALL_SKY_FOV

612000

An estimate of the FOV of the pixel can be calculated from the resolution of the camera. It should give a close approximation.

In [9]:
ALL_SKY_FOV_pixel = ALL_SKY_FOV / 1280
ALL_SKY_FOV_pixel

478.125

## Rotation of earth in arcsecond of a degree per temporal millisecond 

Single Rotation in arcseconds of a degree = $\dfrac{360^{\circ}}{2\pi}$ * 60$\dfrac{arcminutes}{1^{\circ}}$ * 60  $\dfrac{arcseconds}{arcminute}$

Single Rotation in temporal seconds = 24 hr * 60 $\dfrac{min}{hr}$ * 60 $\dfrac{seconds}{min}$ 

The actual rotation in seconds is based on sidereal rate (23 hr 56 min 4s). It's the whole reason we have leap years. For now I'm going to ignore the siderealrate. Because of this the time we calculate 

Single Rotation in sidereal seconds = ((23 hr * 60 $\dfrac{min}{hr}$ + 56 min) * 60 $\dfrac{seconds}{min}$) + 4 seconds 

Rotation of earth in arcseconds per second =  $\dfrac{arcseconds}{second}$

In [10]:
rotation_arcseconds = (360/(2*math.pi)) * 60 * 60
rotation_arcseconds

206264.80624709633

In [11]:
rotation_seconds = 24 * 60 * 60
rotation_seconds

86400

In [12]:
sidereal_rotation_seconds = (((23 * 60)+56) * 60) +4
sidereal_rotation_seconds

86164

In [13]:
rotation_rate = rotation_arcseconds/sidereal_rotation_seconds
rotation_rate

2.393862938664597

So a star on the celestrial equator will have a length of 2.4 arc seconds and a star at the axis of rotation will not move.

## Calculating max exposure time

The idea is to calculate the maximum exposure time based on the star's declination (The distance in degrees from the celestial equator). 

The maximum exposure time is the amount of time to hold a point of light in a particular pixel in the CCD.

We do this to optimize the color obsorption. The longer we can expose a pixel to the light the less atmospheric noise will matter. These effects could be light clouds, thermal effects or other phenomena. You are hoping to average out a better signal to noise.


### Why does a stars declination matter?

The Polaris (north star) is at 89° 15′ 50.8″ while Regulus is at 11° 58′ 01.95″. The distance a star moves over time relative to a pixel on the camera is based on it's declination from the celestrial equator. Polaris barely moves while Regulus while will appear to move farther or hav a greater arc acorss the sky.

Or in other words Polaris can have a longer exposure time than Regulus.

In other words:
declination in degrees = ((((degree * 60$\dfrac{arcminutes}{1^{\circ}}$) + arcminutes) * 60  $\dfrac{arcseconds}{arcminute}$)+ arcseconds) * $\dfrac{1^{\circ}}{3600 arcseconds}$

stars expexted movement (arcseconds) = rotational rate * $\dfrac{90 - \lvert declination \rvert}{90}$

In [14]:
# Polaris (north star) is at 89° 15′ 50.8″ 
degrees = 89
minutes = 15
seconds = 50.8

DEC = ((((degrees*60)+minutes)*60)+seconds)/3600
DEC

89.2641111111111

In [15]:
polaris_movement_per_second = rotation_rate*((90-abs(DEC))/90)
polaris_movement_per_second

0.01957352375651326

In [16]:
# Regulus (brightest in Leo) is at 11° 58′ 1.95″ 
degrees = 11
minutes = 58
seconds = 1.95

DEC = ((((degrees*60)+minutes)*60)+seconds)/3600
DEC

11.967208333333332

In [17]:
regulus_movement_per_second = rotation_rate*((90-abs(DEC))/90)
regulus_movement_per_second

2.0755534219040994

As you can see Regulus will move 2 arcseconds while Polaris will barely move across the sky. 

### So what is the max exposure time?

So depending on a cameras field of view per pixel and the expected movement of a star in seconds you should be able to identify a theoretical max exposure time.

max exposure time = $\dfrac{FOV}{movement per second}$ 

In [18]:
# Max exposure for ASI482MC on Polaris in seconds
max_exposure = ASI482MC_FOV_pixel/polaris_movement_per_second
print(f'(ASI482MC) Polaris exposure in seconds: {max_exposure}')

(ASI482MC) Polaris exposure in seconds: 38.20006719009485


In [19]:
# Max exposure for ASI482MC on Regulus in milliseconds
max_exposure = ASI482MC_FOV_pixel/regulus_movement_per_second * 1000
print(f'(ASI482MC) Regulus exposure in milliseconds: {max_exposure}')

(ASI482MC) Regulus exposure in milliseconds: 360.2460504050915


In [20]:
# Max exposure for ASI120MC-S on Polaris in seconds
max_exposure = ASI120MC_S_FOV_pixel/polaris_movement_per_second
print(f'(ASI120MC-S) Polaris exposure in seconds: {max_exposure}')

(ASI120MC-S) Polaris exposure in seconds: 24.698319303940632


In [21]:
# Max exposure for ASI120MC-S on Regulus in milliseconds
max_exposure = ASI120MC_S_FOV_pixel/regulus_movement_per_second * 1000
print(f'(ASI120MC-S) Regulus exposure in milliseconds: {max_exposure}')

(ASI120MC-S) Regulus exposure in milliseconds: 232.91770500329187


We can also calculate the best exposure time for the "all sky" attachment to the ASI120MC-S. Because it will contain some star on the celestial equator; you can use the rotational rate a 0 Declination. 

In [22]:
max_exposure = ALL_SKY_FOV_pixel/rotation_rate
print(f'(ASI120MC-S) All sky exposure in seconds: {max_exposure}')

(ASI120MC-S) All sky exposure in seconds: 199.72948002892736


# Warning: These are theoretical limits meant to get you near max exposure time. Being a little less should help maximize results.